In [ ]:
!pip install ortools
#!pip install dtale

In [ ]:
import pandas as pd
import numpy as np
#from ortools.sat.python import cp_model
#from dtale import show
#import dtale.app as dtale_app
#dtale_app.USE_COLAB = True

In [ ]:
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 44 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (44/44), done.
/content/cloned-repo
 CP.ipynb  'item master.xlsx'   orders.xlsx   README.md


In [ ]:
df=pd.read_excel('orders.xlsx')

In [ ]:
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']

In [ ]:
show(df,ignore_duplicate=True)

https://pi6pg94wws-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/3

In [ ]:
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']

In [ ]:
df

,SO,FG Code,Order Qty,Plant,due,sodate,Key
0,17023287,gd29904300iX,486.0,1024,223,146,17023287gd29904300iX
1,17023287,gd29907800iX,216.0,1024,223,146,17023287gd29907800iX
2,17023287,gd39000800iX,360.0,1024,223,146,17023287gd39000800iX
3,17023287,gd55102400iX,180.0,1024,223,146,17023287gd55102400iX
4,17023572,gd43208500iX,756.0,1024,225,170,17023572gd43208500iX
...,...,...,...,...,...,...,...
191,17023468,gd28007800iX,336.0,1024,307,246,17023468gd28007800iX
192,17023468,gd28001700iX,640.0,1024,307,246,17023468gd28001700iX
193,17023833,gd55001700iX,320.0,1024,307,245,17023833gd55001700iX
194,17023833,gd55102400iX,360.0,1024,307,245,17023833gd55102400iX


In [ ]:
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [ ]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]

In [20]:
df2=df[['Key','FG Code','Order Qty']]
df2=df2.merge(pa[["FG Code","Line","Output"]], on="FG Code")
df2["duration"]=df2["Order Qty"]/df2["Output"]
del df2["Order Qty"]

In [ ]:
horizon = max(df["due"])

In [ ]:
model = cp_model.CpModel()
model.Minimize(delay)

In [ ]:
delay = {}
for j in jobs:
    for m in mc:
        suffix = '_%s_%i' % (j,m)
        start = model.NewIntVar(0, horizon, "start"+suffix)
        end = model.NewIntVar(0, horizon, "end"+suffix)
        duration = model.newIntervalVar(start, duration[j,m], end, "duration"+suffix)
        delay.append(abs(due[j]-end))

In [ ]:
model.Minimize(delay)

In [ ]:
solver = cp_model.CpSolver()
status = solver.Solve(model)